In [6]:
from sklearn import svm, datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

In [7]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
X = np.array([(-1,1),(1,-1),(1,1),(0,0)])
y = np.array([1,1,1,-1])
loo = LeaveOneOut()
loo.get_n_splits(X)
print(loo)
for k in range(1, 4):
  for i, (train_index, test_index) in enumerate(loo.split(X)):
      kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(X[train_index])
      print(k, accuracy_score(y[test_index], kmeans.predict(X[test_index])))

LeaveOneOut()
1 0.0
1 0.0
1 0.0
1 0.0
2 0.0
2 0.0
2 0.0
2 0.0
3 1.0
3 1.0
3 0.0
3 0.0


In [8]:
import pandas as pd

In [161]:
train = pd.read_csv('/content/Data_train.csv')
test = pd.read_csv('/content/Data_test.csv')

In [162]:
test

,type,group,education,meal,preparation course
0,wild,group D,some college,standard,none
1,wild,group C,master's degree,reduced,completed
2,wild,group B,some college,reduced,completed
3,wild,group A,associate's degree,reduced,none
4,domestic,group B,some college,standard,completed
...,...,...,...,...,...
295,domestic,group B,some college,reduced,none
296,domestic,group C,master's degree,standard,none
297,wild,group D,high school,reduced,none
298,domestic,group C,some high school,reduced,none


In [163]:
train

,type,group,education,meal,preparation course,score-1,score-2,score-3
0,wild,group B,some high school,standard,completed,63,67,67
1,domestic,group C,bachelor's degree,standard,none,67,69,75
2,domestic,group C,some college,standard,completed,69,90,88
3,domestic,group B,some high school,standard,none,62,64,66
4,wild,group E,some college,reduced,none,93,90,83
...,...,...,...,...,...,...,...,...
695,wild,group C,some high school,standard,completed,67,73,68
696,domestic,group B,high school,reduced,none,8,24,23
697,wild,group E,some college,standard,completed,85,75,68
698,wild,group E,associate's degree,standard,none,53,45,40


In [164]:
test.group.unique()

array(['group D', 'group C', 'group B', 'group A', 'group E', nan],
      dtype=object)

In [165]:
train.dtypes

,0
type,object
group,object
education,object
meal,object
preparation course,object
score-1,int64
score-2,int64
score-3,int64


In [166]:
train.columns[train.isna().any()].tolist()

['group']

In [167]:
test.columns[test.isna().any()].tolist()

['group']

In [168]:
train.isna().sum()

,0
type,0
group,20
education,0
meal,0
preparation course,0
score-1,0
score-2,0
score-3,0


In [169]:
test.isna().sum()

,0
type,0
group,12
education,0
meal,0
preparation course,0


In [170]:
train['group'] = train['group'].fillna('F')

In [171]:
test['group'] = test['group'].fillna('F')

In [172]:
def check(row):
  return int(row['score-1'] > 50 and row['score-2'] > 50 and row['score-3'] > 50)

train['Pass'] = train[['score-1', 'score-2', 'score-3']].apply(check, axis=1)

In [173]:
train['Pass'].value_counts()

,count
Pass,
1,555
0,145


In [85]:
filtered = train[train['type'] == 'wild']['Pass']

In [89]:
len(filtered[filtered == 1]) / len(train)

0.38285714285714284

In [90]:
filtered = train[train['type'] == 'domestic']['Pass']
len(filtered[filtered == 1]) / len(train)

0.41

In [204]:
0.41 - 0.38285714285714284

0.027142857142857135

In [106]:
len(train[(train['education'] == 'high school') & (train['Pass'] == 0)])

35

In [104]:
len(train[(train['Pass'] == 1) & (train['type'] == 'wild') & (train['preparation course'] == 'none')])

152

In [99]:
train['score-1'].median()

66.0

In [107]:
filtered = train[(train['type'] == 'domestic') & (train['preparation course'] == 'none')]
Q1 = filtered['score-3'].quantile(0.25)
Q3 = filtered['score-3'].quantile(0.75)
IQR = Q3 - Q1
IQR

np.float64(20.0)

In [108]:
train

,type,group,education,meal,preparation course,score-1,score-2,score-3,Pass
0,wild,group B,some high school,standard,completed,63,67,67,1
1,domestic,group C,bachelor's degree,standard,none,67,69,75,1
2,domestic,group C,some college,standard,completed,69,90,88,1
3,domestic,group B,some high school,standard,none,62,64,66,1
4,wild,group E,some college,reduced,none,93,90,83,1
...,...,...,...,...,...,...,...,...,...
695,wild,group C,some high school,standard,completed,67,73,68,1
696,domestic,group B,high school,reduced,none,8,24,23,0
697,wild,group E,some college,standard,completed,85,75,68,1
698,wild,group E,associate's degree,standard,none,53,45,40,0


In [110]:
train = pd.get_dummies(train, columns=['type', 'group', 'education', 'meal', 'preparation course'])

In [114]:
len(train.columns) - 4

18

In [113]:
test = pd.get_dummies(test, columns=['type', 'group', 'education', 'meal', 'preparation course'])

In [115]:
from sklearn.tree import DecisionTreeClassifier

In [120]:
X = train.drop(['score-1', 'score-2', 'score-3', 'Pass'], axis=1)
y = train['Pass']

In [117]:
m = DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [126]:
from sklearn.model_selection import cross_val_score, GridSearchCV
scores = cross_val_score(
    m, X, y, cv=3, scoring='roc_auc')

In [127]:
np.mean(scores)

np.float64(0.6612555157198013)

In [129]:
parameters = {'max_depth':[2, 20]}
m = DecisionTreeClassifier(criterion='entropy')
clf = GridSearchCV(m, parameters, cv=3, scoring='roc_auc')
clf.fit(X,y)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(criterion='entropy'),
             param_grid={'max_depth': [2, 20]}, scoring='roc_auc')

In [130]:
train

,score-1,score-2,score-3,Pass,type_domestic,type_wild,group_F,group_group A,group_group B,group_group C,...,education_associate's degree,education_bachelor's degree,education_high school,education_master's degree,education_some college,education_some high school,meal_reduced,meal_standard,preparation course_completed,preparation course_none
0,63,67,67,1,False,True,False,False,True,False,...,False,False,False,False,False,True,False,True,True,False
1,67,69,75,1,True,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,True
2,69,90,88,1,True,False,False,False,False,True,...,False,False,False,False,True,False,False,True,True,False
3,62,64,66,1,True,False,False,False,True,False,...,False,False,False,False,False,True,False,True,False,True
4,93,90,83,1,False,True,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,67,73,68,1,False,True,False,False,False,True,...,False,False,False,False,False,True,False,True,True,False
696,8,24,23,0,True,False,False,False,True,False,...,False,False,True,False,False,False,True,False,False,True
697,85,75,68,1,False,True,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
698,53,45,40,0,False,True,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True


In [144]:
train

,type,group,education,meal,preparation course,score-1,score-2,score-3,Pass
0,wild,group B,some high school,standard,completed,63,67,67,1
1,domestic,group C,bachelor's degree,standard,none,67,69,75,1
2,domestic,group C,some college,standard,completed,69,90,88,1
3,domestic,group B,some high school,standard,none,62,64,66,1
4,wild,group E,some college,reduced,none,93,90,83,1
...,...,...,...,...,...,...,...,...,...
695,wild,group C,some high school,standard,completed,67,73,68,1
696,domestic,group B,high school,reduced,none,8,24,23,0
697,wild,group E,some college,standard,completed,85,75,68,1
698,wild,group E,associate's degree,standard,none,53,45,40,0


In [147]:
def bio(row):
    return f"({row['type']},{row['group']})"
train['cat_bio'] = train.apply(bio, axis=1)

In [148]:
train['cat_bio']

,cat_bio
0,"(wild,group B)"
1,"(domestic,group C)"
2,"(domestic,group C)"
3,"(domestic,group B)"
4,"(wild,group E)"
...,...
695,"(wild,group C)"
696,"(domestic,group B)"
697,"(wild,group E)"
698,"(wild,group E)"


In [149]:
train = train[['cat_bio', 'education', 'meal', 'preparation course','Pass']]

In [151]:
train = pd.get_dummies(train, ['cat_bio', 'education', 'meal', 'preparation course'])

In [152]:
train

,Pass,"cat_bio_(domestic,F)","cat_bio_(domestic,group A)","cat_bio_(domestic,group B)","cat_bio_(domestic,group C)","cat_bio_(domestic,group D)","cat_bio_(domestic,group E)","cat_bio_(wild,F)","cat_bio_(wild,group A)","cat_bio_(wild,group B)",...,education_associate's degree,education_bachelor's degree,education_high school,education_master's degree,education_some college,education_some high school,meal_reduced,meal_standard,preparation course_completed,preparation course_none
0,1,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,True,True,False
1,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,True
2,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
3,1,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,True
4,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,True,False
696,0,False,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,True
697,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,True,False
698,0,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,True


In [153]:
m = DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [154]:
X = train.drop(['Pass'], axis=1)
y = train['Pass']

In [155]:
m.fit(X,y)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [158]:
scores = cross_val_score(
    m, X, y, cv=3, scoring='roc_auc')

In [159]:
np.mean(scores)

np.float64(0.676355564748422)

In [160]:
%%capture
!pip install catboost

In [174]:
train = train.drop(['score-1', 'score-2', 'score-3'], axis=1)

In [178]:
X = train.drop(['Pass'], axis=1)
y = train['Pass']

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [201]:
from catboost import CatBoostClassifier

cat_features = [0,1,2,3,4]
model = CatBoostClassifier(iterations=120,
                           learning_rate=0.01,
                           depth=2,
                           eval_metric='AUC')
# Fit model
model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=cat_features, use_best_model=True)

0:	test: 0.7086570	best: 0.7086570 (0)	total: 626us	remaining: 74.5ms
1:	test: 0.6997780	best: 0.7086570 (0)	total: 1.47ms	remaining: 86.9ms
2:	test: 0.7255827	best: 0.7255827 (2)	total: 1.96ms	remaining: 76.5ms
3:	test: 0.6997780	best: 0.7255827 (2)	total: 2.38ms	remaining: 69ms
4:	test: 0.6997780	best: 0.7255827 (2)	total: 3.91ms	remaining: 90ms
5:	test: 0.6997780	best: 0.7255827 (2)	total: 4.35ms	remaining: 82.6ms
6:	test: 0.6997780	best: 0.7255827 (2)	total: 5.72ms	remaining: 92.3ms
7:	test: 0.6997780	best: 0.7255827 (2)	total: 6.13ms	remaining: 85.8ms
8:	test: 0.6997780	best: 0.7255827 (2)	total: 7.61ms	remaining: 93.8ms
9:	test: 0.6997780	best: 0.7255827 (2)	total: 8.03ms	remaining: 88.3ms
10:	test: 0.6997780	best: 0.7255827 (2)	total: 9.4ms	remaining: 93.2ms
11:	test: 0.7031077	best: 0.7255827 (2)	total: 9.81ms	remaining: 88.3ms
12:	test: 0.7255827	best: 0.7255827 (2)	total: 11.3ms	remaining: 93.1ms
13:	test: 0.7255827	best: 0.7255827 (2)	total: 11.7ms	remaining: 88.7ms
14:	test

In [203]:
with open('result.txt', 'a', encoding='utf-8') as the_file:
    for i in model.predict(test):
        the_file.write(f'{i}\n')